# Titanic

**Titanic** alebo **RMS Titanic** (*RMS* je skratka pre *Royal Mail Ship* – *Kráľovský poštový parník*) bol luxusný zaoceánsky parník triedy *Olympic* patriaci spoločnosti *White Star Line*. Jeho úlohou mal byť prevoz cestujúcich a pošty medzi Európou a Severnou Amerikou. Kapacita lode dovoľovala nalodenie *2 453* až *2 603* cestujúcich, batožiny a obmedzený počet kočiarov alebo automobilov. O prevádzku lode a o pohodlie cestujúcich sa staralo *885* až *899* členov posádky. 

![Titanic](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/RMS_Titanic_3.jpg/1024px-RMS_Titanic_3.jpg)

Titanic však stroskotal už počas svojej prvej plavby v noci zo *14.* na *15. apríla* roku *1912*. Zahynulo okolo *1 450* cestujúcich a členov posádky.

V tomto labe sa pozrieme na katastrofu cez čísla. Dataset je ukradnutý zo stránok [Kaggle](https://www.kaggle.com/competitions/titanic), kde slúži ako vstupné zadanie do sveta strojového učenia.

## 1. Základné informácie

V tomto kroku sa pozrieme na to, kto sa vlastne na *Titaniku* viezol v kritickú a vlastne jedinú plavbu.

### 1.1 Vytvorte dataset

Vytvorte dataset so zoznamom všetkých pasažierov. Tento dataset zostavte z dvoch datasetov s názvom `test.csv` a `train.csv`, ktoré sa nachádzajú uložené v priečinku `data/titanic/`. Index zostavte zo stĺpca `PassengerId`.

Výsledný dataset uložte do premennej `passengers`.

In [2]:
# solution


### 1.2 Koľko bolo cestujúcich?

Zistite, koľko cestujúcich sa zúčastnilo plavby.

In [3]:
# solution


### 1.2 Koľko bolo na palube žien a koľko mužov?

Zistite, koľko bolo na palube žien a mužov. Výsledok reprezentujte vo forme slovníka v tvare:

```json
{
    'males': 123,
    'females': 456
}
```

In [4]:
# solution


### 1.3 Koľko bolo na palube detí?

Za dieťa považujeme osobu, ktorá nedosiahla vek *15* rokov.

In [5]:
# solution


### 1.4 Najstarší pasažier

Ktorý z cestujúcich bol najstarším pasažierom? Okrem najstaršieho cestujúceho celkovo vypíšte aj najstaršiu cestujúcu a najstaršieho cestujúceho. Vždy o cestujúcom vypíšte meno, vek a pohlavie.

In [6]:
# solution


### 1.5 Najmladší pasažier

A naopak - ktorý z cestujúcich bol najmladší? Okrem najmladšieho cestujúceho celkovo vypíšte aj najmladšiu cestujúcu a najmladšieho cestujúceho. Vždy vypíšte meno, vek a pohlavie a výsledok vráťte ako objekt typu `Series`.

In [7]:
# solution


### 1.6 Aký bol priemerný vek pasažierov?

Už vieme, kto bol najmladší a kto bol najstarší, tak ešte zistime, aký bol priemerný vek pasažierov na lodi. Výsledok zaokrúhlite na 2 desatinné miesta.

In [8]:
# solution


### 1.7 Zisk z predaja lístkov

Koľko peňazí tvoril zisk z plavby, ktorý sa vyzbieral z cestovného?

In [9]:
# solution


## Nalodenie

### 2.1 Počet prístavov

Z koľkých prístavov nastupovali pasažieri? Vypíšte číslo zodpovedajúce počtu prístavov.

In [10]:
# solution


### 2.2 Prehľad nalodených pasažierov

Vytvorte prehľad, z ktorého prístavu koľko ľudí nastúpilo na palubu. Výsledný objekt typu `Series` nech má ako index názvy prístavov, z ktorých sa pasažieri naloďovali.

In [11]:
# solution


### 2.3 Najväčší prístav

A z ktorého prístavu nastúpilo najviac pasažierov? Vypíšte jeho názov a počet cestujúcich.

In [12]:
# solution


### 2.4 Najviac detí

Z ktorého prístavu nastúpilo najviac detí? Vypíšte jeho názov a počet detí.

In [13]:
# solution


## O preživších

O tom, kto prežil, máme tiež záznamy. Jednak nás budú zaujímať absolútne čísla, ale druhak nás bude zaujímať aj to, čo všetko mohlo mať vplyv na to, že dotyčný človek prežil. A či nemali vyššiu pravdepodobnosť prežitia tí, ktorí patrili do vyššej vrstvy alebo boli mladší.

### 3.1 Koľko ľudí katastrofu prežilo?

Zistite celkový počet ľudí, ktorí prežili potopenie *Titanic*-u. Okrem celkového počtu uveďte tento počet aj percentuálne.

In [14]:
# solution


### 3.2 Preživší a ich pohlavie

Malo na prežitie vplyv aj pohlavie? Koľko prežilo vo výsledku mužov a koľko žien? A koľko to bolo z celkového počtu mužov a žien?


In [15]:
# solution


### 3.3 Preživší a ich vek

Mal na prežitie vplyv aj vek pasažierov? Koľko prežilo vo výsledku detí a koľko dospelých? A koľko to bolo z celkového počtu detí a dospelých?

In [16]:
# solution


### 3.4 Preživší a ich trieda

Mala na prežitie vplyv aj spoločenská trieda pasažierov? Koľko prežilo vo výsledku chudákov a koľko zámožných pasažierov? A koľko to bolo z celkového počtu cestujúcich?

In [17]:
# solution
